In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) * n_conv2 


In [ ]:
weights = {
    'wc1': tf.Variable(tf.random.normal([conv1_k, conv1_k, input_channels, n_conv1])),
    'wc1': tf.Variable(tf.random.normal([conv2_k, conv2_k, n_conv1, n_conv2])),
    'wh1': tf.Variable(tf.random.normal([input_size_to_hidden, n_hidden])),
    'wo': tf.Variable(tf.random.normal([n_hidden, n_out]))
}

biases = {
    'bc1': tf.Variable(tf.random.normal([n_conv1])),
    'bc2': tf.Variable(tf.random.normal([n_conv2])),
    'bh1': tf.Variable(tf.random.normal([n_hidden])),
    'bo': tf.Variable(tf.random.normal([n_out]))
}

In [ ]:
def conv(x, weights, bias, strides=1):
    out = tf.nn.conv2d(x, weights, padding='SAME', strides = [1, strides, strides, 1])
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k=2):
    return tf.nn.max_pool(x, padding='SAME', ksize=[1, k, k, 1], strides=[1, k, k, 1])

In [ ]:
def cnn(x, weights, biases):
    x = tf.reshape(x, shape=[-1, input_height, input_width, input_channels])
    conv1 = conv(x, weights['wc1'], biases['bc1'], stride_conv1)
    conv1_pool = maxpooling(conv1, max_pool1_k)
    
    conv2 = conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv2)
    conv2_pool = maxpooling(conv2, max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool, shape=[-1, input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['wh1']), biases['bh1'])
    hidden_output = tf.nn.relu(hidden_output_before_activation)
    
    output = tf.add(tf.matmul(hidden_output, weights['wo']), biases['bo'])
    return output

In [ ]:

x = tf.reshape(x, shape[-1, input_height, input_width, input_channels])
conv1 = conv(x, weights['wc1'], biases['bc1'], stride_conv1)
print(maxpooling(conv1, 3))

In [ ]:
x = tf.compat.v1.placeholder('float', [None, input_pixels])
y = tf.compat.v1.placeholder(tf.int32, [None, n_out])
pred = cnn(x, weights, biases)

In [ ]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred , labels=y))

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
optimize = optimizer.minimize(cost)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())